# AIMO3 Turbo-Strap V4 (PROMETHEUS)
## Unknown Knowns Extracted & Operationalized

**V4 Novel Features:**
1. **Problem Classifier** - Route to optimal strategy ensemble BEFORE inference
2. **Kolmogorov Weighting** - Shorter code = higher confidence (Occam's Razor)
3. **Oracle Enumeration** - If space <10^6, just brute force
4. **Causal Grammar Extraction** - Parse constraints from problem text
5. **Adaptive Temperature** - Start high, anneal on consensus detection
6. **Time Budget Decay** - Exponential allocation (hard problems get more)
7. **Entropy Sentinel** - Early stop on confidence spike

In [1]:
%pip uninstall --yes "tensorflow" "matplotlib" "keras" "scikit-learn" 2>/dev/null || true

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: matplotlib 3.7.2
Uninstalling matplotlib-3.7.2:
  Successfully uninstalled matplotlib-3.7.2
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
assert torch.cuda.is_available(), "GPU NOT ENABLED"
print(f"✓ GPU: {torch.cuda.get_device_name(0)} | VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")

✓ GPU: NVIDIA H100 80GB HBM3 | VRAM: 85.3GB


In [3]:
import os, sys, gc, time, random, warnings, re, zlib, statistics, tempfile, subprocess, math
from collections import Counter, defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Optional, List, Dict, Any, Tuple, Set
from dataclasses import dataclass
from enum import Enum
import numpy as np
import pandas as pd
import polars as pl

sys.setrecursionlimit(20000)
warnings.simplefilter('ignore')
os.environ.update({"TRANSFORMERS_NO_TF":"1","TRANSFORMERS_NO_FLAX":"1","TRITON_PTXAS_PATH":"/usr/local/cuda/bin/ptxas","CUDA_VISIBLE_DEVICES":"0","TOKENIZERS_PARALLELISM":"false"})

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

from transformers import set_seed
from vllm import LLM, SamplingParams
import kaggle_evaluation.aimo_3_inference_server
set_seed(SEED)

START_TIME = time.time()
TOTAL_BUDGET = (4*60+40)*60
cutoff_time = START_TIME + TOTAL_BUDGET
AAR_LOG = defaultdict(int)
PROBLEM_COUNT = 0
SOLVED_COUNT = 0

print(f"✓ Environment ready. Budget: {TOTAL_BUDGET//60}min")

INFO 12-06 04:29:30 [__init__.py:216] Automatically detected platform cuda.
✓ Environment ready. Budget: 280min


In [4]:
# ═══════════════════════════════════════════════════════════════════════════════
# PROBLEM CLASSIFIER (Unknown Known #1)
# Route to optimal strategy ensemble BEFORE inference
# ═══════════════════════════════════════════════════════════════════════════════

class ProblemType(Enum):
    NUMBER_THEORY = "nt"      # Modular arithmetic, divisibility, primes
    COMBINATORICS = "comb"    # Counting, arrangements, probability
    GEOMETRY = "geom"         # Shapes, coordinates, angles
    ALGEBRA = "alg"           # Equations, polynomials, sequences
    MIXED = "mixed"           # Multi-domain or unclear

@dataclass
class ProblemProfile:
    primary_type: ProblemType
    secondary_type: Optional[ProblemType]
    constraints: List[str]       # Extracted constraints
    objective: str               # What to find
    enumeration_hint: Optional[int]  # If small space, estimate size
    modulo_target: Optional[int] # If answer needs mod N
    confidence: float            # Classification confidence

def classify_problem(question: str) -> ProblemProfile:
    """Extract causal grammar and classify problem type."""
    q = question.lower()
    
    # Scoring for each type
    scores = {t: 0.0 for t in ProblemType}
    
    # NUMBER THEORY signals
    nt_keywords = ['remainder', 'modulo', 'mod ', 'divisible', 'prime', 'gcd', 'lcm', 
                   'factor', 'digit', 'divides', 'coprime', 'congruent']
    for kw in nt_keywords:
        if kw in q: scores[ProblemType.NUMBER_THEORY] += 2
    
    # COMBINATORICS signals
    comb_keywords = ['how many', 'count', 'number of ways', 'arrangements', 'permutation',
                     'combination', 'choose', 'select', 'probability', 'subset', 'partition']
    for kw in comb_keywords:
        if kw in q: scores[ProblemType.COMBINATORICS] += 2
    
    # GEOMETRY signals
    geom_keywords = ['triangle', 'circle', 'square', 'rectangle', 'polygon', 'angle',
                     'area', 'perimeter', 'diameter', 'radius', 'point', 'line', 'plane',
                     'perpendicular', 'parallel', 'tangent', 'inscribed', 'circumscribed']
    for kw in geom_keywords:
        if kw in q: scores[ProblemType.GEOMETRY] += 2
    
    # ALGEBRA signals
    alg_keywords = ['equation', 'polynomial', 'root', 'solve', 'sequence', 'series',
                    'function', 'variable', 'expression', 'inequality', 'sum of', 'product of']
    for kw in alg_keywords:
        if kw in q: scores[ProblemType.ALGEBRA] += 2
    
    # Sort by score
    sorted_types = sorted(scores.items(), key=lambda x: -x[1])
    primary = sorted_types[0][0]
    secondary = sorted_types[1][0] if sorted_types[1][1] > 2 else None
    
    # If no clear winner, mark as MIXED
    if sorted_types[0][1] < 2:
        primary = ProblemType.MIXED
    
    # Extract constraints ("such that", "where", "given that")
    constraints = []
    constraint_patterns = [
        r'such that ([^.]+)',
        r'where ([^.]+)',
        r'given that ([^.]+)',
        r'with ([^.]+)',
        r'satisf(?:y|ies) ([^.]+)',
    ]
    for pattern in constraint_patterns:
        matches = re.findall(pattern, q)
        constraints.extend(matches)
    
    # Extract objective ("find", "compute", "determine", "what is")
    objective = "unknown"
    obj_patterns = [
        r'find (?:the )?(.+?)(?:\.|$)',
        r'compute (?:the )?(.+?)(?:\.|$)',
        r'determine (?:the )?(.+?)(?:\.|$)',
        r'what is (?:the )?(.+?)(?:\?|$)',
        r'calculate (?:the )?(.+?)(?:\.|$)',
    ]
    for pattern in obj_patterns:
        match = re.search(pattern, q)
        if match:
            objective = match.group(1)[:100]
            break
    
    # Detect enumeration hint (small bounded space)
    enum_hint = None
    numbers = [int(x) for x in re.findall(r'\b(\d+)\b', question) if 1 < int(x) < 10000]
    if numbers:
        # Heuristic: if largest number < 1000 and we're counting, space is likely enumerable
        if max(numbers) < 1000 and primary == ProblemType.COMBINATORICS:
            enum_hint = max(numbers) ** 2  # Rough upper bound
    
    # Detect modulo target
    mod_target = None
    mod_match = re.search(r'(?:mod|modulo|remainder when divided by)\s*(\d+)', q)
    if mod_match:
        mod_target = int(mod_match.group(1))
    elif 'mod 1000' in q or 'modulo 1000' in q:
        mod_target = 1000
    
    confidence = min(1.0, sorted_types[0][1] / 10.0)
    
    return ProblemProfile(
        primary_type=primary,
        secondary_type=secondary,
        constraints=constraints,
        objective=objective,
        enumeration_hint=enum_hint,
        modulo_target=mod_target,
        confidence=confidence
    )

print("✓ Problem Classifier loaded")

✓ Problem Classifier loaded


In [5]:
# Config
IS_SAMPLE = pd.read_csv("/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv").shape[0] == 3
LLM_MODEL_PATH = '/kaggle/input/gpt-oss-20b/transformers/default/1'

# Strategy weights by problem type (tuned ensemble)
STRATEGY_WEIGHTS = {
    ProblemType.NUMBER_THEORY: {'pot_sympy': 3, 'pot_algorithmic': 2, 'pot_bruteforce': 1},
    ProblemType.COMBINATORICS: {'pot_bruteforce': 3, 'pot_algorithmic': 2, 'pot_constraint': 1},
    ProblemType.GEOMETRY: {'pot_sympy': 2, 'pot_algorithmic': 2, 'pot_constraint': 2},
    ProblemType.ALGEBRA: {'pot_sympy': 3, 'pot_algorithmic': 2, 'pot_verification': 1},
    ProblemType.MIXED: {'pot_algorithmic': 2, 'pot_sympy': 2, 'pot_bruteforce': 1, 'pot_constraint': 1},
}

# Adaptive parameters
BASE_SAMPLES = 10
CONSENSUS_THRESHOLD = 0.5
BASIN_THRESHOLD = 0.02
NCD_GROK_THRESHOLD = 0.35
PANIC_TIME = 300
ORACLE_THRESHOLD = 1_000_000  # If enum space < this, brute force

# Temperature annealing
TEMP_START = 0.7
TEMP_MIN = 0.3
TEMP_DECAY = 0.85

# Kolmogorov constants
KOLMOGOROV_WEIGHT = 0.15  # How much to weight by inverse code length

PHI = (1 + 5**0.5) / 2
FIBONACCI = [1,1,2,3,5,8,13,21,34,55,89,144,233,377,610,987,1597,2584,4181,6765]

print(f"✓ Config: Sample={IS_SAMPLE}")

✓ Config: Sample=True


In [6]:
# Model Load
MAX_NUM_SEQS = 32
MAX_MODEL_LEN = 16384

model_kwargs = {
    "max_num_seqs": MAX_NUM_SEQS, "max_model_len": MAX_MODEL_LEN,
    "trust_remote_code": True, "tensor_parallel_size": 1,
    "gpu_memory_utilization": 0.95, "seed": SEED, "enable_prefix_caching": True,
}

try:
    llm = LLM(LLM_MODEL_PATH, kv_cache_dtype="fp8", enforce_eager=False, **model_kwargs)
    print("✓ Model (FP8 KV)")
except:
    llm = LLM(LLM_MODEL_PATH, **model_kwargs)
    print("✓ Model (standard)")

tokenizer = llm.get_tokenizer()

INFO 12-06 04:29:40 [utils.py:328] non-default args: {'trust_remote_code': True, 'kv_cache_dtype': 'fp8', 'seed': 42, 'max_model_len': 16384, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.95, 'max_num_seqs': 32, 'disable_log_stats': True, 'model': '/kaggle/input/gpt-oss-20b/transformers/default/1'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 12-06 04:30:15 [__init__.py:742] Resolved architecture: GptOssForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


ERROR 12-06 04:30:15 [config.py:278] Error retrieving safetensors: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/input/gpt-oss-20b/transformers/default/1'. Use `repo_type` argument if needed., retrying 1 of 2
ERROR 12-06 04:30:17 [config.py:276] Error retrieving safetensors: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/input/gpt-oss-20b/transformers/default/1'. Use `repo_type` argument if needed.
INFO 12-06 04:30:17 [__init__.py:2764] Downcasting torch.float32 to torch.bfloat16.
INFO 12-06 04:30:17 [__init__.py:1815] Using max model len 16384
WARNING 12-06 04:30:21 [_ipex_ops.py:16] Import error msg: No module named 'intel_extension_for_pytorch'
WARNING 12-06 04:30:21 [__init__.py:1217] mxfp4 quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-06 04:30:21 [cache.py:174] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. Meanwhile

...

[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=188) INFO 12-06 04:30:57 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=188) WARNING 12-06 04:30:57 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=188) INFO 12-06 04:30:57 [gpu_model_runner.py:2338

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:33<01:07, 33.58s/it]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [01:04<00:32, 32.28s/it]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [01:32<00:00, 30.08s/it]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [01:32<00:00, 30.81s/it]
(EngineCore_DP0 pid=188) 


(EngineCore_DP0 pid=188) INFO 12-06 04:32:31 [default_loader.py:268] Loading weights took 92.48 seconds
(EngineCore_DP0 pid=188) WARNING 12-06 04:32:31 [marlin_utils_fp4.py:196] Your GPU does not have native support for FP4 computation but FP4 quantization is being used. Weight-only FP4 compression will be used leveraging the Marlin kernel. This may degrade performance for compute-heavy workloads.
(EngineCore_DP0 pid=188) INFO 12-06 04:32:32 [gpu_model_runner.py:2392] Model loading took 13.7164 GiB and 93.615974 seconds
(EngineCore_DP0 pid=188) INFO 12-06 04:32:44 [backends.py:539] Using cache directory: /root/.cache/vllm/torch_compile_cache/7161b16b9d/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=188) INFO 12-06 04:32:44 [backends.py:550] Dynamo bytecode transform time: 12.19 s
(EngineCore_DP0 pid=188) INFO 12-06 04:32:48 [backends.py:194] Cache the graph for dynamic shape for later use
(EngineCore_DP0 pid=188) INFO 12-06 04:33:12 [backends.py:215] Compiling a graph f

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 83/83 [00:04<00:00, 18.85it/s]


(EngineCore_DP0 pid=188) INFO 12-06 04:33:21 [gpu_model_runner.py:3118] Graph capturing finished in 5 secs, took 0.94 GiB
(EngineCore_DP0 pid=188) INFO 12-06 04:33:21 [gpu_worker.py:391] Free memory on device (78.84/79.44 GiB) on startup. Desired GPU memory utilization is (0.95, 75.47 GiB). Actual usage is 13.72 GiB for weight, 1.69 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.94 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=63251194777` to fit into requested memory, or `--kv-cache-memory=66871069184` to fully utilize gpu memory. Current kv cache memory in use is 64417211289 bytes.
(EngineCore_DP0 pid=188) INFO 12-06 04:33:21 [core.py:218] init engine (profile, create kv cache, warmup model) took 49.09 seconds
INFO 12-06 04:33:22 [llm.py:295] Supported_tasks: ['generate']
INFO 12-06 04:33:22 [__init__.py:36] No IOProcessor plugins requested by the model
✓ Model (FP8 KV)


In [7]:
# Enhanced Stdlib
STDLIB = '''
import sys; sys.setrecursionlimit(20000)
import math, numpy as np
from itertools import *
from collections import *
from functools import lru_cache, reduce
from fractions import Fraction
try:
    import sympy
    from sympy import *
    from sympy import Rational, Integer, symbols, solve, simplify, sqrt, Abs, binomial
    from sympy.ntheory import factorint, divisors, totient, mobius, primefactors, isprime
    from sympy.ntheory.modular import crt as sympy_crt
    from sympy.solvers.diophantine import diophantine
    from sympy.geometry import Point, Line, Circle, Triangle, Polygon
except: pass

def is_prime(n):
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0: return False
    return True

def prime_sieve(n):
    if n < 2: return []
    sieve = [True] * (n + 1)
    sieve[0] = sieve[1] = False
    for i in range(2, int(n**0.5) + 1):
        if sieve[i]:
            for j in range(i*i, n + 1, i):
                sieve[j] = False
    return [i for i, v in enumerate(sieve) if v]

def prime_factors(n):
    factors = []
    d = 2
    while d * d <= n:
        while n % d == 0: factors.append(d); n //= d
        d += 1
    if n > 1: factors.append(n)
    return factors

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def lcm(a, b): return a * b // gcd(a, b)

def extended_gcd(a, b):
    if a == 0: return b, 0, 1
    g, x, y = extended_gcd(b % a, a)
    return g, y - (b // a) * x, x

def mod_inverse(a, m):
    g, x, _ = extended_gcd(a % m, m)
    return x % m if g == 1 else None

def crt(remainders, moduli):
    M = reduce(lambda x, y: x * y, moduli)
    x = 0
    for r, m in zip(remainders, moduli):
        Mi = M // m
        inv = mod_inverse(Mi, m)
        if inv is None: return None
        x += r * Mi * inv
    return x % M

def shoelace_area(coords):
    n = len(coords)
    area = sum(coords[i][0] * coords[(i+1)%n][1] - coords[(i+1)%n][0] * coords[i][1] for i in range(n))
    return abs(area) / 2

def dist(p1, p2): return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def C(n, k):
    if k < 0 or k > n: return 0
    if k == 0 or k == n: return 1
    return math.factorial(n) // (math.factorial(k) * math.factorial(n - k))

def P(n, k): return math.factorial(n) // math.factorial(n - k) if k <= n else 0
'''

SNOOP = '''
_vars = dict(globals())
for _v in ['answer', 'result', 'ans', 'res', 'final', 'output', 'solution', 'total', 'count']:
    if _v in _vars and _vars[_v] is not None:
        try: print(int(_vars[_v])); break
        except: pass
'''

print(f"✓ Stdlib ({len(STDLIB)} chars)")

✓ Stdlib (2351 chars)


In [8]:
# ═══════════════════════════════════════════════════════════════════════════════
# KOLMOGOROV WEIGHTING (Unknown Known #2)
# Shorter code = higher confidence (Occam's Razor operationalized)
# ═══════════════════════════════════════════════════════════════════════════════

@dataclass
class AnswerCandidate:
    value: int
    code: str
    strategy: str
    code_length: int
    kolmogorov_weight: float

def compute_kolmogorov_weight(code: str) -> float:
    """Inverse complexity weight. Shorter = better."""
    if not code: return 0.5
    # Normalize by typical code length (500 chars)
    normalized_len = len(code) / 500.0
    # Sigmoid-ish: short code gets weight ~1.5, long code ~0.5
    return 1.0 / (0.5 + normalized_len * KOLMOGOROV_WEIGHT)

def relative_distance(a: float, b: float) -> float:
    if a == b: return 0.0
    if a == 0 or b == 0: return 1.0
    return abs(a - b) / max(abs(a), abs(b))

def normalize_answer(raw: Any) -> Optional[int]:
    if raw is None: return None
    try:
        s = str(raw).replace(',', '').replace(' ', '').strip()
        s = re.sub(r'[^0-9.eE+-]', '', s)
        if not s: return None
        val = float(s)
        if val != val or abs(val) == float('inf'): return None
        result = int(round(val))
        if result < 0: result = abs(result)
        return result % 100000
    except: return None

def benford_proximity(n: int) -> float:
    if n <= 0: return 1000
    fib_dist = min(abs(n - f) for f in FIBONACCI if f > 0)
    phi_dist = min(abs(n - int(PHI ** k)) for k in range(1, 20) if PHI ** k < 100000)
    round_score = -50 if n % 100 == 0 else (-20 if n % 10 == 0 else 0)
    return min(fib_dist, phi_dist) + round_score

def select_answer_with_kolmogorov(candidates: List[AnswerCandidate], 
                                   mod_target: Optional[int] = None) -> int:
    """CIC Basin Clustering + Kolmogorov Weighting + Integration Force."""
    if not candidates: return 0
    
    # Apply modulo if specified
    if mod_target:
        for c in candidates:
            c.value = c.value % mod_target
    
    # Group into basins
    values = sorted(set(c.value for c in candidates))
    basins = defaultdict(list)
    
    for c in candidates:
        # Find basin by proximity
        assigned = False
        for bval in basins.keys():
            if relative_distance(c.value, bval) < BASIN_THRESHOLD:
                basins[bval].append(c)
                assigned = True
                break
        if not assigned:
            basins[c.value].append(c)
    
    # Score basins
    basin_scores = []
    for bval, members in basins.items():
        # Mass (count)
        mass = len(members)
        
        # Kolmogorov weight (sum of inverse complexities)
        k_weight = sum(c.kolmogorov_weight for c in members)
        
        # Integration force (strategy diversity)
        strategies = set(c.strategy for c in members)
        integration = 1.5 if len(strategies) > 1 else 1.0
        
        # Combined score
        score = mass * k_weight * integration
        
        # Median value of basin
        median_val = int(statistics.median([c.value for c in members]))
        
        basin_scores.append((score, median_val, members))
    
    basin_scores.sort(key=lambda x: -x[0])
    
    # Tiebreaker: Benford proximity
    if len(basin_scores) >= 2 and basin_scores[0][0] < basin_scores[1][0] * 1.1:
        top2 = basin_scores[:2]
        top2.sort(key=lambda x: benford_proximity(x[1]))
        AAR_LOG['benford_tiebreak'] += 1
        return max(0, min(top2[0][1], 99999))
    
    final = basin_scores[0][1]
    
    # 0/1 sanity check
    if final in (0, 1) and len(candidates) > 3:
        non_trivial = [c for c in candidates if c.value not in (0, 1)]
        if len(non_trivial) >= 2:
            return select_answer_with_kolmogorov(non_trivial, mod_target)
    
    return max(0, min(final, 99999))

print("✓ Kolmogorov selection loaded")

✓ Kolmogorov selection loaded


In [9]:
# Code Execution

def extract_python_code(text: str) -> Optional[str]:
    for pattern in [r'```python\s*\n(.*?)```', r'```py\s*\n(.*?)```', r'```\s*\n(.*?)```']:
        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
        if matches: return matches[-1].strip()
    return None

def calc_timeout(code: str) -> int:
    base = 10
    loops = len(re.findall(r'\bfor\b|\bwhile\b', code))
    if loops >= 2: base += 10
    if any(kw in code for kw in ['itertools', 'permutations', 'combinations', 'product']): base += 10
    if '@lru_cache' in code: base += 5
    return min(base, 30)

def self_heal(code: str, stderr: str) -> Optional[str]:
    healed = code
    if 'NameError' in stderr:
        missing = re.findall(r"name '(\w+)' is not defined", stderr)
        for name in missing:
            if name in ['np', 'numpy']: healed = 'import numpy as np\n' + healed
            elif name in ['math', 'sqrt', 'gcd']: healed = 'import math\nfrom math import *\n' + healed
            elif name in ['sympy', 'Symbol', 'solve']: healed = 'from sympy import *\n' + healed
            elif name in ['Counter', 'defaultdict']: healed = 'from collections import *\n' + healed
            elif name in ['combinations', 'permutations']: healed = 'from itertools import *\n' + healed
    if 'RecursionError' in stderr: healed = 'import sys; sys.setrecursionlimit(50000)\n' + healed
    return healed if healed != code else None

def execute_code(code: str, retries: int = 2) -> Tuple[Optional[int], str]:
    if not code: return None, ""
    has_print = 'print(' in code
    full = STDLIB + "\n" + code + ("" if has_print else "\n" + SNOOP)
    timeout = calc_timeout(code)
    
    for attempt in range(retries + 1):
        try:
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
                f.write(full); f.flush()
                result = subprocess.run(['python', f.name], capture_output=True, text=True, timeout=timeout)
                os.unlink(f.name)
                
                if result.returncode == 0 and result.stdout.strip():
                    numbers = re.findall(r'-?\d+\.?\d*', result.stdout)
                    if numbers: return normalize_answer(numbers[-1]), code
                
                if result.stderr and attempt < retries:
                    healed = self_heal(code, result.stderr)
                    if healed:
                        AAR_LOG['self_heal'] += 1
                        full = STDLIB + "\n" + healed + ("" if has_print else "\n" + SNOOP)
                        code = healed
                        continue
                if 'Error' in str(result.stderr): AAR_LOG['runtime_error'] += 1
        except subprocess.TimeoutExpired: AAR_LOG['timeout'] += 1
        except: AAR_LOG['exec_exception'] += 1
        break
    return None, code

def extract_boxed(text: str) -> Optional[int]:
    for pattern in [r'\\boxed\{([^}]+)\}', r'boxed\{([^}]+)\}', r'[Aa]nswer\s*(?:is|=|:)\s*([0-9]+)']:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches: return normalize_answer(matches[-1])
    return None

print("✓ Execution engine loaded")

✓ Execution engine loaded


In [10]:
# ═══════════════════════════════════════════════════════════════════════════════
# TYPE-SPECIFIC PROMPTS (Causal Grammar Injection)
# ═══════════════════════════════════════════════════════════════════════════════

STRATEGY_PROMPTS = {
    'pot_algorithmic': (
        "You are an algorithmic mathematician. Write self-contained Python. "
        "Start with imports. Use assert to validate. Handle edge cases. "
        "Use int() not float. Use pow(a,b,m) for modular exp. Print ONE integer."
    ),
    'pot_sympy': (
        "You are a symbolic computation expert. Use sympy for EXACT arithmetic. "
        "Use Rational(a,b) NOT a/b. Never use floats. Verify with simplify(). "
        "Print ONE integer."
    ),
    'pot_bruteforce': (
        "You are a computational enumerator. Systematically enumerate. "
        "Budget: 10^7 iterations max. Use itertools, memoization, early termination. "
        "Assert bounds before loops. Print ONE integer."
    ),
    'pot_constraint': (
        "You are a constraint satisfaction expert. Model as CSP. "
        "List ALL constraints first. Propagate. Backtrack on violation. Print ONE integer."
    ),
    'pot_verification': (
        "You are a verification mathematician. Solve TWO different ways. "
        "Compare. Only print when BOTH agree. Print ONE integer."
    ),
}

TYPE_HINTS = {
    ProblemType.NUMBER_THEORY: "\n\nHINT: This is a NUMBER THEORY problem. Consider: modular arithmetic, divisibility, prime factorization, Chinese Remainder Theorem, Fermat's Little Theorem.",
    ProblemType.COMBINATORICS: "\n\nHINT: This is a COMBINATORICS problem. Consider: counting principles, binomial coefficients, inclusion-exclusion, generating functions, recursion with memoization.",
    ProblemType.GEOMETRY: "\n\nHINT: This is a GEOMETRY problem. Consider: coordinate geometry, distance formula, shoelace formula for area, similar triangles, trigonometry.",
    ProblemType.ALGEBRA: "\n\nHINT: This is an ALGEBRA problem. Consider: polynomial manipulation, Vieta's formulas, sequences and series, solving systems of equations.",
    ProblemType.MIXED: "",
}

def create_prompt(question: str, strategy: str, profile: ProblemProfile, 
                  prev_wrong: Optional[int] = None, temp_idx: int = 0) -> Tuple[List[Dict], str]:
    """Create prompt with causal grammar injection."""
    
    system = STRATEGY_PROMPTS.get(strategy, STRATEGY_PROMPTS['pot_algorithmic'])
    system += " Enclose code in ```python ... ```."
    
    # Inject type hint
    type_hint = TYPE_HINTS.get(profile.primary_type, "")
    
    # Inject extracted constraints
    constraint_injection = ""
    if profile.constraints:
        constraint_injection = f"\n\nEXTRACTED CONSTRAINTS: {'; '.join(profile.constraints[:3])}"
    
    # Inject modulo target
    mod_injection = ""
    if profile.modulo_target:
        mod_injection = f"\n\nIMPORTANT: Final answer must be mod {profile.modulo_target}."
    
    # Ratcheting injection
    ratchet = ""
    if prev_wrong is not None:
        ratchet = f"\n\nWARNING: A previous attempt got {prev_wrong}, which was WRONG. Use a different approach."
    
    user = f"Problem: {question}{type_hint}{constraint_injection}{mod_injection}{ratchet}\n\nWrite Python. Print ONLY the integer answer."
    
    return [{"role": "system", "content": system}, {"role": "user", "content": user}], strategy

def create_cot_prompt(question: str, profile: ProblemProfile) -> Tuple[List[Dict], str]:
    system = "You are a mathematical oracle. Think step-by-step. ATTACK your solution before finalizing. Only output \\boxed{answer} at 95%+ confidence."
    type_hint = TYPE_HINTS.get(profile.primary_type, "")
    user = f"{question}{type_hint}\n\nSolve completely. Final integer in \\boxed{{}}."
    return [{"role": "system", "content": system}, {"role": "user", "content": user}], "cot"

print("✓ Prompts loaded")

✓ Prompts loaded


In [11]:
# Batch Generation with Adaptive Temperature

POT_STOP = ["```output", "```\nOutput", "# Output", "\n\n\n"]
COT_STOP = ["</think>", "\n\nQuestion:"]

def batch_generate(messages: List[List[Dict]], mode: str = 'pot', temp: float = 0.6) -> List[str]:
    prompts = [tokenizer.apply_chat_template(m, tokenize=False, add_generation_prompt=True) for m in messages]
    params = SamplingParams(
        temperature=temp, top_p=0.9, min_p=0.05, max_tokens=8192,
        skip_special_tokens=True, stop=POT_STOP if mode == 'pot' else COT_STOP
    )
    outputs = llm.generate(prompts, sampling_params=params)
    return [o.outputs[0].text for o in outputs]

def process_pot(outputs: List[str], strategies: List[str]) -> List[AnswerCandidate]:
    candidates = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {}
        for i, out in enumerate(outputs):
            code = extract_python_code(out)
            if code:
                futures[executor.submit(execute_code, code)] = (i, code, strategies[i] if i < len(strategies) else 'pot')
        
        for future in as_completed(futures):
            i, code, strat = futures[future]
            try:
                ans, final_code = future.result()
                if ans is not None:
                    candidates.append(AnswerCandidate(
                        value=ans,
                        code=final_code,
                        strategy=strat,
                        code_length=len(final_code),
                        kolmogorov_weight=compute_kolmogorov_weight(final_code)
                    ))
            except: pass
    return candidates

def process_cot(outputs: List[str]) -> List[AnswerCandidate]:
    candidates = []
    for out in outputs:
        ans = extract_boxed(out)
        if ans is not None:
            candidates.append(AnswerCandidate(
                value=ans, code="", strategy="cot",
                code_length=0, kolmogorov_weight=1.0  # No code = neutral weight
            ))
    return candidates

print("✓ Batch processing loaded")

✓ Batch processing loaded


In [12]:
# ═══════════════════════════════════════════════════════════════════════════════
# ORACLE ENUMERATION (Unknown Known #3)
# If space is small enough, just brute force in code
# ═══════════════════════════════════════════════════════════════════════════════

def try_oracle_enumeration(question: str, profile: ProblemProfile) -> Optional[int]:
    """For small enumeration spaces, just brute force."""
    if profile.enumeration_hint is None or profile.enumeration_hint > ORACLE_THRESHOLD:
        return None
    
    AAR_LOG['oracle_attempt'] += 1
    
    # Generate a brute-force enumeration prompt
    oracle_prompt = f"""Problem: {question}

The solution space appears small (estimated {profile.enumeration_hint} possibilities).
Write Python that EXHAUSTIVELY ENUMERATES all possibilities and checks each one.
Do NOT try to be clever. Just iterate through ALL cases.
Print the answer."""
    
    system = "You are a brute-force enumerator. Write code that checks EVERY possibility. No optimization needed - just exhaustive search. Enclose in ```python ... ```."
    msgs = [[{"role": "system", "content": system}, {"role": "user", "content": oracle_prompt}]]
    
    outputs = batch_generate(msgs, 'pot', temp=0.3)  # Low temp for deterministic
    candidates = process_pot(outputs, ['oracle'])
    
    if candidates:
        AAR_LOG['oracle_success'] += 1
        return candidates[0].value
    return None

print("✓ Oracle enumeration loaded")

✓ Oracle enumeration loaded


In [13]:
# Panic Mode

def panic_guess(question: str, profile: ProblemProfile) -> int:
    AAR_LOG['panic'] += 1
    q = question.lower()
    numbers = [int(x) for x in re.findall(r'\b\d+\b', question) if 0 < int(x) < 100000]
    
    if profile.modulo_target:
        if numbers: return numbers[0] % profile.modulo_target
        return 0
    
    if 'how many' in q:
        return min(numbers) if numbers else 1
    if 'sum' in q or 'total' in q:
        small = [n for n in numbers if n < 1000]
        return sum(small) % 100000 if small else 0
    
    return numbers[0] % 100000 if numbers else 0

print("✓ Panic mode loaded")

✓ Panic mode loaded


In [14]:
# ═══════════════════════════════════════════════════════════════════════════════
# MAIN SOLVER with Time Budget Decay + Adaptive Temperature
# ═══════════════════════════════════════════════════════════════════════════════

def predict_for_question(question: str) -> int:
    global PROBLEM_COUNT, SOLVED_COUNT
    PROBLEM_COUNT += 1
    
    time_remaining = cutoff_time - time.time()
    if time_remaining < PANIC_TIME:
        profile = classify_problem(question)
        print(f"⚠️ PANIC ({time_remaining:.0f}s)")
        return panic_guess(question, profile)
    
    # ═══════════════════════════════════════════════════════════════════════════
    # PHASE 0: CLASSIFY PROBLEM
    # ═══════════════════════════════════════════════════════════════════════════
    profile = classify_problem(question)
    print(f"Q: {question[:60]}...")
    print(f"  Type: {profile.primary_type.value} (conf={profile.confidence:.2f}) | Mod: {profile.modulo_target} | Enum: {profile.enumeration_hint}")
    
    # ═══════════════════════════════════════════════════════════════════════════
    # PHASE 0.5: TRY ORACLE ENUMERATION
    # ═══════════════════════════════════════════════════════════════════════════
    if profile.enumeration_hint and profile.enumeration_hint < ORACLE_THRESHOLD:
        oracle_ans = try_oracle_enumeration(question, profile)
        if oracle_ans is not None:
            print(f"  ✓ Oracle: {oracle_ans}")
            SOLVED_COUNT += 1
            return oracle_ans
    
    # Get strategy weights for this problem type
    weights = STRATEGY_WEIGHTS.get(profile.primary_type, STRATEGY_WEIGHTS[ProblemType.MIXED])
    
    # Build weighted strategy list
    strategies = []
    for strat, weight in weights.items():
        strategies.extend([strat] * weight)
    
    all_candidates: List[AnswerCandidate] = []
    current_temp = TEMP_START
    
    # ═══════════════════════════════════════════════════════════════════════════
    # PHASE 1: INITIAL SCOUTING
    # ═══════════════════════════════════════════════════════════════════════════
    n_phase1 = min(len(strategies), 6)
    print(f"  Phase 1: PoT x{n_phase1} (temp={current_temp:.2f})")
    
    msgs_p1, strats_p1 = [], []
    for i in range(n_phase1):
        strat = strategies[i % len(strategies)]
        m, s = create_prompt(question, strat, profile, temp_idx=i)
        msgs_p1.append(m)
        strats_p1.append(s)
    
    outputs_p1 = batch_generate(msgs_p1, 'pot', temp=current_temp)
    candidates_p1 = process_pot(outputs_p1, strats_p1)
    all_candidates.extend(candidates_p1)
    
    answers_p1 = [c.value for c in candidates_p1]
    print(f"    → {len(answers_p1)} answers: {answers_p1}")
    
    # EARLY STOP CHECK
    if len(answers_p1) >= 2:
        counts = Counter(answers_p1)
        best, count = counts.most_common(1)[0]
        if count / len(answers_p1) >= CONSENSUS_THRESHOLD:
            print(f"  ✓ Consensus: {best}")
            SOLVED_COUNT += 1
            return int(best)
    
    # ═══════════════════════════════════════════════════════════════════════════
    # PHASE 2: ADAPTIVE EXPANSION (Temperature Annealing)
    # ═══════════════════════════════════════════════════════════════════════════
    if time.time() > cutoff_time - 120:
        if all_candidates:
            return select_answer_with_kolmogorov(all_candidates, profile.modulo_target)
        return panic_guess(question, profile)
    
    # Anneal temperature
    current_temp = max(TEMP_MIN, current_temp * TEMP_DECAY)
    
    # Get most common wrong answer for ratcheting
    prev_wrong = None
    if answers_p1:
        counts = Counter(answers_p1)
        if len(counts) > 1:
            prev_wrong = counts.most_common(1)[0][0]
    
    if len(candidates_p1) > 0:
        # PoT working → expand with different strategies
        n_phase2 = 4
        print(f"  Phase 2: PoT expand x{n_phase2} (temp={current_temp:.2f})")
        
        msgs_p2, strats_p2 = [], []
        for i in range(n_phase2):
            strat = strategies[(i + n_phase1) % len(strategies)]
            m, s = create_prompt(question, strat, profile, prev_wrong=prev_wrong, temp_idx=i+10)
            msgs_p2.append(m)
            strats_p2.append(s)
        
        outputs_p2 = batch_generate(msgs_p2, 'pot', temp=current_temp)
        candidates_p2 = process_pot(outputs_p2, strats_p2)
        all_candidates.extend(candidates_p2)
        print(f"    → {len(candidates_p2)} answers: {[c.value for c in candidates_p2]}")
    else:
        # PoT failed → CoT fallback
        print(f"  Phase 2: CoT fallback x4")
        AAR_LOG['cot_fallback'] += 1
        
        msgs_p2 = [create_cot_prompt(question, profile)[0] for _ in range(4)]
        outputs_p2 = batch_generate(msgs_p2, 'cot', temp=current_temp)
        candidates_p2 = process_cot(outputs_p2)
        all_candidates.extend(candidates_p2)
        print(f"    → {len(candidates_p2)} answers: {[c.value for c in candidates_p2]}")
    
    # ═══════════════════════════════════════════════════════════════════════════
    # FINAL: KOLMOGOROV-WEIGHTED BASIN SELECTION
    # ═══════════════════════════════════════════════════════════════════════════
    if not all_candidates:
        print("  ✗ No answers")
        AAR_LOG['no_answers'] += 1
        return panic_guess(question, profile)
    
    final = select_answer_with_kolmogorov(all_candidates, profile.modulo_target)
    
    # Log distribution
    all_values = [c.value for c in all_candidates]
    print(f"  ★ Final: {final} | {Counter(all_values)}")
    
    SOLVED_COUNT += 1
    gc.collect()
    torch.cuda.empty_cache()
    return final

print("✓ Solver loaded")

✓ Solver loaded


In [15]:
# Kaggle Interface

def predict(id_: pl.DataFrame, question: pl.DataFrame, answer: Optional[pl.DataFrame] = None) -> pl.DataFrame:
    id_val, question_str = id_.item(0), question.item(0)
    time_left = (cutoff_time - time.time()) / 60
    print(f"\n{'='*60}\nID: {id_val} | Time: {time_left:.1f}m | Solved: {SOLVED_COUNT}/{PROBLEM_COUNT}")
    
    prediction = predict_for_question(question_str)
    
    print(f"ANSWER: {prediction}")
    print(f"AAR: {dict(AAR_LOG)}")
    print(f"{'='*60}")
    
    return pl.DataFrame({'id': id_val, 'answer': prediction})

print("✓ Interface loaded")

✓ Interface loaded


In [16]:
# Start Server

inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("🚀 Competition mode")
    inference_server.serve()
else:
    print("🧪 Local test")
    inference_server.run_local_gateway(('/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv',))
    print(f"\n{'='*60}\nFINAL AAR:")
    for k, v in sorted(AAR_LOG.items()): print(f"  {k}: {v}")
    print(f"Solved: {SOLVED_COUNT}/{PROBLEM_COUNT}")

🧪 Local test

ID: 111bbb | Time: 276.3m | Solved: 0/0
Q: What is $0\times10$?...
  Type: mixed (conf=0.00) | Mod: None | Enum: None
  Phase 1: PoT x6 (temp=0.70)


Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

    → 6 answers: [0, 0, 0, 0, 0, 0]
  ✓ Consensus: 0
ANSWER: 0
AAR: {}

ID: 000aaa | Time: 276.2m | Solved: 1/1
Q: What is $1-1$?...
  Type: mixed (conf=0.00) | Mod: None | Enum: None
  Phase 1: PoT x6 (temp=0.70)


Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

    → 5 answers: [0, 0, 0, 0, 0]
  ✓ Consensus: 0
ANSWER: 0
AAR: {}

ID: 222ccc | Time: 276.1m | Solved: 2/2
Q: Solve $4+x=4$ for $x$....
  Type: alg (conf=0.20) | Mod: None | Enum: None
  Phase 1: PoT x6 (temp=0.70)


Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

    → 4 answers: [0, 0, 0, 0]
  ✓ Consensus: 0
ANSWER: 0
AAR: {}

FINAL AAR:
Solved: 3/3
